# Daily Fantasy Football Projection comparison

In [1]:
import numpy as np
import pandas as pd
import requests

from pymongo import MongoClient
import pprint

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import json
import time

import matplotlib.pyplot as plt
# from matplotlib import pyplot as plt

%matplotlib inline
plt.style.use('ggplot')


## Read html


In [2]:
df = pd.read_html('https://www.footballdiehards.com/fantasyfootball/dailygames/Draftkings-Salary-data.cfm', header = 1)

# Turn list into DatFrame
salary = df[0].dropna(axis=0, thresh=4)

In [3]:
salary

sort_sal = salary.sort_values(by = 'SALARY', ascending = True)

sort_sal

,Player,Pos,year,week,SALARY,Score,Factor,Rank
68,"Detroit, Lions",DST,2020,14,$2000,4.3,2.2,30
619,"New York Jets, Jets",DST,2020,14,$2100,-,-,999
618,"Jacksonville, Jaguars",DST,2020,14,$2200,-,-,999
103,"Philadelphia, Eagles",DST,2020,14,$2200,4.3,2.0,29
90,"Buffalo, Bills",DST,2020,14,$2300,4.6,2.0,25
...,...,...,...,...,...,...,...,...
123,"Hill, Tyreek",WR,2020,14,$8500,15.8,1.9,3
2,"Henry, Derrick",RB,2020,14,$8700,24.6,2.8,1
18,"McCaffrey, Christian",RB,2020,14,$9200,24.2,2.6,2
94,"Adams, Davante",WR,2020,14,$9300,18.4,2.0,1


In [4]:
#renaming player name column to just name
salary = salary.rename(columns={'Player': 'name'})

# reversing name to combine with player projection data
salary['name'] = salary['name'].str.replace(r'(.+),\s+(.+)', r'\2 \1')

#Converting SALARY to a float type and dropping $

salary['SALARY'] = salary['SALARY'].str.replace('$', '')
salary['SALARY'] = salary['SALARY'].astype('float')

# importing and cleaning QB df

In [5]:

#Reading in QB DF
qb = pd.read_csv('Fantasy_proj/qb.csv')
qb = qb.dropna()


# drop row 2020 OUTLOOK column that just contains string
qb = qb.drop(['2020 OUTLOOK'], axis = 1)




In [6]:
#rename the Player Name row to match Salary df
qb = qb.rename(columns={'PLAYER NAME': 'name'})
qb = qb.rename(columns={'PLAYER POSITION': 'Pos'})

#drop pointless row
#qb= qb.drop(['Unnamed: 0'], axis = 1)

type(qb['name'])

pandas.core.series.Series

In [7]:
#create a single dataframe with QBs from salary added to QB for fantasy projections

qb_sal = pd.merge(qb, salary, how='left', left_on='name', right_on='name')

#qb_sal


In [8]:
#Leaving Out Rows Where SALARY = NaN = Means a Player is not playing this week

qb_sal = qb_sal[qb_sal['SALARY'].notna()]

#qb_sal

In [9]:
# Filtered out more rows where player Score does not == '-', 
# Also mean they wont get much playing time or wont play at all


qb_sal =qb_sal[qb_sal.Score != '-']

#convert $ value into float in 2 step first replace $ then convert to float

# qb_sal['SALARY'] = qb_sal['SALARY'].str.replace('$', '')
# qb_sal['SALARY'] = qb_sal['SALARY'].astype('float')


#Final qb list including their salary filtered to only players playing
qb_sal

,Unnamed: 0,name,TEAM NAME,Pos_x,2019 COMPLETED PASS,2019 ATTEMPTED PASS,2019 PASSING YDS,2019 PASSING TD,2019 INT,2019 CAR,...,2020 RUSH YDS,2020 RUSH TD,2020 FPTS,Pos_y,year,week,SALARY,Score,Factor,Rank
0,1,Lamar Jackson,Ravens,QB,265.0,401.0,3127.0,36.0,6.0,176.0,...,902.0,6.0,338.23,QB,2020,14,7000.0,17.4,2.5,6
1,2,Patrick Mahomes,Chiefs,QB,319.0,484.0,4031.0,26.0,5.0,43.0,...,280.0,3.0,332.66,QB,2020,14,8100.0,18.8,2.3,3
2,3,Deshaun Watson,Texans,QB,333.0,495.0,3852.0,26.0,12.0,82.0,...,519.0,4.0,314.06,QB,2020,14,7600.0,16.7,2.2,7
4,5,Russell Wilson,Seahawks,QB,341.0,516.0,4110.0,31.0,5.0,75.0,...,397.0,2.0,293.96,QB,2020,14,7900.0,22.5,2.9,1
5,6,Kyler Murray,Cardinals,QB,349.0,542.0,3722.0,20.0,12.0,93.0,...,469.0,3.0,293.94,QB,2020,14,7200.0,16.4,2.3,8
6,7,Tom Brady,Buccaneers,QB,373.0,613.0,4057.0,24.0,8.0,26.0,...,27.0,2.0,291.22,QB,2020,14,6900.0,15.2,2.2,10
7,8,Matt Ryan,Falcons,QB,408.0,616.0,4466.0,26.0,14.0,34.0,...,125.0,1.0,282.50,QB,2020,14,5700.0,14.4,2.5,13
8,9,Matthew Stafford,Lions,QB,187.0,291.0,2499.0,19.0,5.0,20.0,...,123.0,1.0,268.80,QB,2020,14,5700.0,14.0,2.5,14
9,10,Carson Wentz,Eagles,QB,388.0,607.0,4039.0,27.0,7.0,62.0,...,213.0,2.0,268.38,QB,2020,14,5100.0,5.6,1.1,34
11,12,Josh Allen,Bills,QB,271.0,461.0,3089.0,20.0,9.0,109.0,...,545.0,5.0,266.26,QB,2020,14,7100.0,15.7,2.2,9


# Cleaning RB DATA

In [10]:
#importing rb fantasy projection & dropping NA
rb = pd.read_csv('Fantasy_proj/rb.csv')
rb = rb.rename(columns={'PLAYER NAME': 'name'})
#rb.dropna()

In [11]:
rb_sal = pd.merge(rb, salary, how='left', left_on='name', right_on='name')

In [12]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

rb_sal.drop(['Pos', 'year', 'week'], axis=1)
rb_sal = rb_sal[rb_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
rb_sal =rb_sal[rb_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
rb_sal =rb_sal[rb_sal['TEAM NAME'] != 'FA']


#drop rows where Salary > certain threshold ... Means they prob wont be playing or very little playing time

#rb_sal = rb_sal[rb_sal['SALARY'] > 5000]

In [13]:
#Leave out rows where Salary is below 5000 threshold== Means they wont get much playing time

rb_sal = rb_sal[rb_sal['SALARY'] > 5000]



# Cleaning WR df

In [14]:
#importing WR projection data and changing name column to combine DF
wr = pd.read_csv('Fantasy_proj/wr.csv')
wr= wr.rename(columns={'PLAYER NAME': 'name'})

In [15]:
#merging WR projections with SALARY

wr_sal = pd.merge(wr, salary, how='left', left_on='name', right_on='name')



In [16]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

wr_sal.drop(['Pos', 'year', 'week', 'Unnamed: 0', '2020 PLAYER OUTLOOK'], axis=1)
wr_sal = wr_sal[wr_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
wr_sal =wr_sal[wr_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
wr_sal =wr_sal[wr_sal['TEAM'] != 'FA']

wr_sal = wr_sal[wr_sal['SALARY'] > 4000]

In [17]:
#Dropping pointless columns

wr_sal.drop(['Pos', 'year', 'week', 'Unnamed: 0', '2020 PLAYER OUTLOOK', '2019 TAR', '2019 REC', '2019 REC YARDS', '2019 AVG REC YARDS', '2019 REC TDS'  ], axis=1)

,name,TEAM,POSITION,2019 RUSH CAR,2019 RUSH YARDS,2019 RUSH TDS,2019 FPTS,2020 TAR,2020 REC,2020 REC YARDS,2020 AVG REC YARDS,2020 REC TDS,2020 RUSH CAR,2020 RUSH YARDS,2020 RUSH TDS,2020 FPTS,SALARY,Score,Factor,Rank
0,Michael Thomas,Saints,WR,1.0,-9.0,0.0,374.60,152.0,114.0,1351.0,11.9,9.0,0.0,0.0,0.0,301.44,7100.0,13.6,1.9,11.0
1,DeAndre Hopkins,Cardinals,WR,2.0,18.0,0.0,268.54,141.0,92.0,1140.0,12.3,8.0,4.0,23.0,0.0,255.30,7600.0,15.3,2.0,6.0
2,Julio Jones,Falcons,WR,2.0,-3.0,0.0,274.10,139.0,89.0,1245.0,14.0,7.0,2.0,10.0,0.0,253.95,6600.0,14.5,2.2,9.0
3,Chris Godwin,Buccaneers,WR,1.0,8.0,0.0,276.10,131.0,88.0,1191.0,13.6,8.0,0.0,0.0,0.0,251.83,6300.0,12.6,2.0,16.0
4,Davante Adams,Packers,WR,0.0,0.0,0.0,212.70,137.0,90.0,1149.0,12.7,7.0,0.0,0.0,0.0,249.39,9300.0,18.4,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Gabriel Davis,Bills,WR,NaN,NaN,NaN,NaN,13.0,7.0,97.0,13.8,0.0,0.0,0.0,0.0,19.70,4300.0,6.7,1.6,57.0
148,Nelson Agholor,Raiders,WR,2.0,7.0,0.0,94.00,10.0,6.0,68.0,10.8,0.0,0.0,0.0,0.0,16.08,4700.0,7.1,1.5,51.0
153,Tim Patrick,Broncos,WR,0.0,0.0,0.0,37.80,8.0,5.0,57.0,12.6,0.0,0.0,0.0,0.0,12.05,4200.0,8.1,1.9,42.0
235,Travis Fulgham,Lions,WR,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,4900.0,4.5,0.9,78.0


# Cleaning TE Data

In [18]:
#importing and changing name column to combine with salary df

te = pd.read_csv('Fantasy_proj/te.csv')
te = te.rename(columns={'PLAYER NAME': 'name'})

In [19]:
#merging te projections with SALARY

te_sal = pd.merge(te, salary, how='left', left_on='name', right_on='name')

te_sal.drop(['Pos', 'year', 'week', 'Unnamed: 0', '2019 TAR', '2020 OUTLOOK' ], axis=1)

,name,TEAM NAME,PLAYER POSITION,2019 REC,2019 REC YDS,2019 AVG REC YDS,2019 REC TD,2019 RUSH CAR,2019 RUSH YDS,2019 RUSH TD,...,2020 AVG REC YDS,2020 REC TD,2020 RUSH CAR,2020 RUSH YDS,2020 RUSH TD,2020 FPTS,SALARY,Score,Factor,Rank
0,Travis Kelce,Chiefs,TE,97.0,1229.0,12.7,5.0,1.0,4.0,1.0,...,12.4,9.0,2.0,6.0,0.0,251.34,7400.0,15.2,2.1,1.0
1,George Kittle,49ers,TE,85.0,1053.0,12.4,5.0,5.0,22.0,0.0,...,12.3,7.0,5.0,17.0,0.0,231.77,5400.0,-,-,999.0
2,Zach Ertz,Eagles,TE,88.0,916.0,10.4,6.0,0.0,0.0,0.0,...,10.5,7.0,0.0,0.0,0.0,201.16,3700.0,4.9,1.3,25.0
3,Mark Andrews,Ravens,TE,64.0,852.0,13.3,10.0,0.0,0.0,0.0,...,13.4,8.0,0.0,0.0,0.0,199.44,5300.0,3.8,0.7,32.0
4,Darren Waller,Raiders,TE,90.0,1145.0,12.7,3.0,2.0,5.0,0.0,...,11.6,5.0,0.0,0.0,0.0,192.05,6800.0,13.4,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Ben Watson,Patriots,TE,17.0,173.0,10.2,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
188,Delanie Walker,Titans,TE,21.0,215.0,10.2,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
189,C.J. Conrad,FA,TE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
190,Bradley Sowell,FA,TE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN


In [20]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

te_sal.drop(['Pos', 'year', 'week'], axis=1)
te_sal = te_sal[te_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
te_sal =te_sal[te_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
te_sal =te_sal[te_sal['TEAM NAME'] != 'FA']

te_sal = te_sal[te_sal['SALARY'] > 5000]

te_sal

,Unnamed: 0,name,TEAM NAME,PLAYER POSITION,2019 TAR,2019 REC,2019 REC YDS,2019 AVG REC YDS,2019 REC TD,2019 RUSH CAR,...,2020 RUSH TD,2020 FPTS,2020 OUTLOOK,Pos,year,week,SALARY,Score,Factor,Rank
0,1,Travis Kelce,Chiefs,TE,136.0,97.0,1229.0,12.7,5.0,1.0,...,0.0,251.34,Kelce enters the 2020 season attempting to pac...,TE,2020.0,14.0,7400.0,15.2,2.1,1.0
3,4,Mark Andrews,Ravens,TE,98.0,64.0,852.0,13.3,10.0,0.0,...,0.0,199.44,"A 2018 third-round pick, Andrews exited his ro...",TE,2020.0,14.0,5300.0,3.8,0.7,32.0
4,5,Darren Waller,Raiders,TE,117.0,90.0,1145.0,12.7,3.0,2.0,...,0.0,192.05,Waller's breakout 2019 season marked one of th...,TE,2020.0,14.0,6800.0,13.4,2.0,2.0
35,36,Taysom Hill,Saints,TE,22.0,19.0,234.0,12.3,6.0,27.0,...,2.0,72.39,"Hill is back in New Orleans and, while technic...",QB,2020.0,14.0,6600.0,15.0,2.3,11.0


# importing and cleaning kicker data

In [21]:
#no kicker salary no kicker in draft Kings
kicker = pd.read_csv('Fantasy_proj/kicker.csv')
kicker = kicker.rename(columns={'PLAYER NAME': 'name'})



#Merging Salary and kicker DF

In [22]:
#no kicker salary no kicker in draft Kings
defense = pd.read_csv('Fantasy_proj/defense.csv')
defense = defense.rename(columns={'PLAYER NAME': 'name'})

type(salary)

pandas.core.frame.DataFrame

In [23]:
#Completed data frames Titles

#qb_sal
#rb_sal
#wr_sal
#te_sal


In [24]:
df = pd.read_html('http://rotoguru1.com/cgi-bin/fyday.pl?week=13&game=dk')

# Turn list into DatFrame
df_ = df[5].dropna(axis=0, thresh=4)

df_

,0,1,2,3,4
0,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
1,Quarterbacks,Team,Opp.,Points,Salary
2,"Carr, Derek",lvr,@ nyj,35.74,"$5,800"
3,"Allen, Josh",buf,@ sfo,34.1,"$7,100"
4,"Mayfield, Baker",cle,@ ten,33.46,"$5,300"
...,...,...,...,...,...
388,San Francisco,sfo,v. buf,2,"$2,500"
389,Arizona,ari,v. lar,2,"$2,900"
390,Denver,den,@ kan,1,"$1,900"
391,Tennessee,ten,v. cle,-2,"$2,900"


In [25]:
new_header = df_[1] #grab the first row for the header
df_ = df_.iloc[2:] #take the data less the header row
#df_.columns = new_header #set the header row as the df header

In [26]:
df_.columns = ['name', 'team', 'opp', 'points', 'salary']

df_

,name,team,opp,points,salary
2,"Carr, Derek",lvr,@ nyj,35.74,"$5,800"
3,"Allen, Josh",buf,@ sfo,34.1,"$7,100"
4,"Mayfield, Baker",cle,@ ten,33.46,"$5,300"
5,"Tannehill, Ryan",ten,v. cle,31.56,"$6,200"
6,"Stafford, Matthew",det,@ chi,30.08,"$5,700"
...,...,...,...,...,...
388,San Francisco,sfo,v. buf,2,"$2,500"
389,Arizona,ari,v. lar,2,"$2,900"
390,Denver,den,@ kan,1,"$1,900"
391,Tennessee,ten,v. cle,-2,"$2,900"


In [27]:
df_filtered = df_[df_['salary'] != 'Jump to: QB | RB | WR | TE | PK | Def |'] 
df_filtered = df_[df_['salary'] != 'Jump to: QB | RB | WR | TE | PK | Def |'] 
df_filtered = df_[df_['salary'] != 'Salary'] 

df_filtered['salary'] = df_filtered['salary'].str.replace('$', '')
df_filtered['salary'] = df_filtered['salary'].str.replace(',', '')
df_filtered['salary'] = df_filtered['salary'].astype(dtype = float, errors='ignore')

df_filtered

/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,name,team,opp,points,salary
2,"Carr, Derek",lvr,@ nyj,35.74,5800
3,"Allen, Josh",buf,@ sfo,34.1,7100
4,"Mayfield, Baker",cle,@ ten,33.46,5300
5,"Tannehill, Ryan",ten,v. cle,31.56,6200
6,"Stafford, Matthew",det,@ chi,30.08,5700
...,...,...,...,...,...
388,San Francisco,sfo,v. buf,2,2500
389,Arizona,ari,v. lar,2,2900
390,Denver,den,@ kan,1,1900
391,Tennessee,ten,v. cle,-2,2900


In [29]:
for el in df_filtered['salary']:
    print(el)

5800
7100
5300
6200
5700
5800
6400
5400
6300
7500
5800
6800
5000
8200
6500
5900
7600
7700
5900
5100
4800
5400
5600
5000
5300
4800
4800
6900
4700
5400
4800
4400
Jump to: QB | RB | WR | TE | PK | Def |
5500
9500
7200
7300
4000
5700
6300
5100
4400
7000
4900
7700
4000
5600
4500
4300
5900
5700
5300
4400
7100
4000
5200
5400
5800
5400
4000
4000
4000
4000
4400
4000
4000
9200
4000
4900
5500
6100
6200
4000
5200
4700
4000
4000
5300
4000
4200
5900
4000
5000
6700
4000
4400
5300
5400
4000
4200
4500
4000
nan
4000
4800
4000
4200
6800
4000
4000
6000
4000
4000
4000
4000
4400
4000
4000
4000
4000
4000
4000
4200
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4400
Jump to: QB | RB | WR | TE | PK | Def |
5100
9000
5000
6900
5700
4300
3500
7000
5100
4000
7300
6500
5400
6200
4200
7800
7500
7200
5900
nan
3000
3000
4500
3300
3600
4300
6600
6100
4500
9100
3400
3000
4700
3000
6300
6700
3000
3600
8200
3300
7400
7600
4500
5600
3000
3100
4600
3000
8100
5800
4900
30

In [58]:
wordy = df_filtered[df_filtered['salary'] == 'Jump to: QB | RB | WR | TE | PK | Def |']

In [60]:
wordy

,name,team,opp,points,salary
34,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
137,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
276,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
361,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
363,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |


In [61]:
df_filtered.reset_index(drop=True)

,name,team,opp,points,salary
0,"Carr, Derek",lvr,@ nyj,35.74,5800
1,"Allen, Josh",buf,@ sfo,34.1,7100
2,"Mayfield, Baker",cle,@ ten,33.46,5300
3,"Tannehill, Ryan",ten,v. cle,31.56,6200
4,"Stafford, Matthew",det,@ chi,30.08,5700
...,...,...,...,...,...
381,San Francisco,sfo,v. buf,2,2500
382,Arizona,ari,v. lar,2,2900
383,Denver,den,@ kan,1,1900
384,Tennessee,ten,v. cle,-2,2900


In [62]:
wordy2 = df_filtered[df_filtered['salary'] == 'Jump to: QB | RB | WR | TE | PK | Def |']

In [63]:
wordy2

,name,team,opp,points,salary
34,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
137,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
276,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
361,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |
363,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |,Jump to: QB | RB | WR | TE | PK | Def |


In [68]:
df_filtered.drop(34)

KeyError: "['34'] not found in axis"

In [141]:
#import CVS file for this weeks fantasy data because old data sucked
draft_kings = pd.read_csv('dksalary.csv')

#Grouping by position
wr = draft_kings[draft_kings.Position == 'WR']
rb = draft_kings[draft_kings.Position == 'RB']
qb= draft_kings[draft_kings.Position == 'QB']
te = draft_kings[draft_kings.Position == 'TE']
defense = draft_kings[draft_kings.Position == 'DST']

qb

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
6,QB,Patrick Mahomes (15904800),Patrick Mahomes,15904800,QB,8100,KC@MIA 12/13/2020 01:00PM ET,KC,28.14
7,QB,Russell Wilson (15904801),Russell Wilson,15904801,QB,7900,NYJ@SEA 12/13/2020 04:05PM ET,SEA,26.46
10,QB,Deshaun Watson (15904802),Deshaun Watson,15904802,QB,7600,HOU@CHI 12/13/2020 01:00PM ET,HOU,25.40
12,QB,Aaron Rodgers (15904803),Aaron Rodgers,15904803,QB,7500,GB@DET 12/13/2020 04:25PM ET,GB,25.32
18,QB,Kyler Murray (15904804),Kyler Murray,15904804,QB,7200,ARI@NYG 12/13/2020 01:00PM ET,ARI,27.50
...,...,...,...,...,...,...,...,...,...
239,QB,Jacob Eason (15904863),Jacob Eason,15904863,QB,4000,IND@LV 12/13/2020 04:05PM ET,IND,0.00
246,QB,Kyle Allen (15904858),Kyle Allen,15904858,QB,4000,WAS@SF 12/13/2020 04:25PM ET,WAS,11.75
252,QB,Nate Sudfeld (15904860),Nate Sudfeld,15904860,QB,4000,NO@PHI 12/13/2020 04:25PM ET,PHI,0.00
253,QB,Trevor Siemian (15904861),Trevor Siemian,15904861,QB,4000,NO@PHI 12/13/2020 04:25PM ET,NO,0.00


#function to filter each by dropping top and bottom 25%
def in_qrange(ser, q):
    return ser.between(*ser.quantile(q=q))

#print (in_qrange(wr["Salary"], q=[0.25, 0.75]))

wr_mid = wr.loc[wr["Salary"].transform(in_qrange, q=[])]

In [160]:
wr_top5 = wr.loc[0:12]

In [161]:
wr_top5

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
1,WR,Davante Adams (15905135),Davante Adams,15905135,WR/FLEX,9300,GB@DET 12/13/2020 04:25PM ET,GB,27.89
4,WR,Tyreek Hill (15905137),Tyreek Hill,15905137,WR/FLEX,8500,KC@MIA 12/13/2020 01:00PM ET,KC,23.66
5,WR,DK Metcalf (15905139),DK Metcalf,15905139,WR/FLEX,8400,NYJ@SEA 12/13/2020 04:05PM ET,SEA,20.24
8,WR,Keenan Allen (15905141),Keenan Allen,15905141,WR/FLEX,7700,ATL@LAC 12/13/2020 04:25PM ET,LAC,19.69
11,WR,DeAndre Hopkins (15905143),DeAndre Hopkins,15905143,WR/FLEX,7600,ARI@NYG 12/13/2020 01:00PM ET,ARI,19.24


In [166]:
qb_top5 = qb.loc[0:18]

In [167]:
qb_top5

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
6,QB,Patrick Mahomes (15904800),Patrick Mahomes,15904800,QB,8100,KC@MIA 12/13/2020 01:00PM ET,KC,28.14
7,QB,Russell Wilson (15904801),Russell Wilson,15904801,QB,7900,NYJ@SEA 12/13/2020 04:05PM ET,SEA,26.46
10,QB,Deshaun Watson (15904802),Deshaun Watson,15904802,QB,7600,HOU@CHI 12/13/2020 01:00PM ET,HOU,25.40
12,QB,Aaron Rodgers (15904803),Aaron Rodgers,15904803,QB,7500,GB@DET 12/13/2020 04:25PM ET,GB,25.32
18,QB,Kyler Murray (15904804),Kyler Murray,15904804,QB,7200,ARI@NYG 12/13/2020 01:00PM ET,ARI,27.50


In [176]:
te_top5 = te.loc[0:130]

In [177]:
te_top5

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
16,TE,Travis Kelce (15905501),Travis Kelce,15905501,TE/FLEX,7400,KC@MIA 12/13/2020 01:00PM ET,KC,21.30
27,TE,Darren Waller (15905503),Darren Waller,15905503,TE/FLEX,6800,IND@LV 12/13/2020 04:05PM ET,LV,16.52
76,TE,George Kittle (15905505),George Kittle,15905505,TE/FLEX,5400,WAS@SF 12/13/2020 04:25PM ET,SF,17.35
102,TE,T.J. Hockenson (15905507),T.J. Hockenson,15905507,TE/FLEX,5000,GB@DET 12/13/2020 04:25PM ET,DET,12.12
119,TE,Rob Gronkowski (15905509),Rob Gronkowski,15905509,TE/FLEX,4800,MIN@TB 12/13/2020 01:00PM ET,TB,9.54


In [184]:
rb_top5 = rb.loc[0:20]

In [185]:
rb_top5

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,RB,Dalvin Cook (15904873),Dalvin Cook,15904873,RB/FLEX,9400,MIN@TB 12/13/2020 01:00PM ET,MIN,26.94
2,RB,Christian McCaffrey (15904875),Christian McCaffrey,15904875,RB/FLEX,9200,DEN@CAR 12/13/2020 01:00PM ET,CAR,30.13
3,RB,Derrick Henry (15904877),Derrick Henry,15904877,RB/FLEX,8700,TEN@JAX 12/13/2020 01:00PM ET,TEN,20.74
9,RB,Aaron Jones (15904879),Aaron Jones,15904879,RB/FLEX,7600,GB@DET 12/13/2020 04:25PM ET,GB,19.93
13,RB,James Robinson (15904881),James Robinson,15904881,RB/FLEX,7500,TEN@JAX 12/13/2020 01:00PM ET,JAX,19.73
20,RB,Alvin Kamara (15904883),Alvin Kamara,15904883,RB/FLEX,7100,NO@PHI 12/13/2020 04:25PM ET,NO,23.65
